In [3]:
%load_ext autoreload
%autoreload 2

# sweep.ipynb

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment


# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 40,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 300

# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
exp = Experiment()
print(f"{exp.exp_dir}")


x_vals = np.linspace(0.15, 0.25, 10, float)
y_vals = np.linspace(0.25, 0.25, 1, float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)
initial_params = np.array([0.20, 0.20], float)

# 5) Optimiser
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

# 6) Sweep different JJ/EPR weightings
w_jj = 0.5
timestamp = datetime.now().strftime("%Y%m%d-%H%M-%S")

# Save the run-level config once
exp.save_config({
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
    "notebook_timestamp": timestamp,
    "random_seed": evaluator.random_seed,
})

filename="results.jsonl"

results = exp.stream_results(
    opt.sweep_multi_objective(param_grid, w_jj=0.5, verbose=True),
    filename="results.jsonl",
    verbose=True
)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Experiment directory created at: experiments\exp_20251026-1040-52

experiments\exp_20251026-1040-52

---------- Starting multi-objective sweep over 10 with w_jj=0.5 ----------

Evaluating point 0/10: [0.15 0.25], w_jj=0.5
jj: 1 sources, loss=-1.501e+13
sa: 40 sources, loss=9.674e+21
ma: 40 sources, loss=-1.796e+21
[0] Saved result to results.jsonl
    loss=4.244302e+21
Evaluating point 1/10: [0.16111111 0.25      ], w_jj=0.5
jj: 1 sources, loss=-2.007e+13
sa: 40 sources, loss=7.409e+21
ma: 40 sources, loss=-1.932e+21
[1] Saved result to results.jsonl
    loss=3.067261e+21
Evaluating point 2/10: [0.17222222 0.25      ], w_jj=0.5
jj: 1 sources, loss=-1.981e+14
sa: 40 sources, loss=8.122e+21
ma: 40 sources, loss=-3.036e+21
[2] Saved result to results.jsonl
    loss=3.059090e+21
Evaluating point 3/10: [0.18333333 0.25      ], w_jj=0.5
jj: 1 sources, loss=-1.053e+15
sa: 40 sources, loss=1.373e+22
ma: 40 

In [18]:

import matplotlib.pyplot as plt
import numpy as np

records = exp.load_jsonl(filename)
print(len(records))
print(records[0])

#automatically sample individual y values.
offset = 0
if len(y_vals) > 1:
    records = records[offset::len(y_vals)]

x = [r['params'][0] for r in records]
grad = [r["grad_jj"][0] *4e-13 for r in records]

# centre loss around zero
mean_loss = np.mean([r["loss_jj"] for r in records])
loss = [r["loss_jj"] - mean_loss for r in records]

def numerical_gradient(x, y):
    ngrad = []
    for i in range(1, len(y)-1):
        dy = y[i+1] - y[i-1]
        dx = x[i+1] - x[i-1]
        ngrad.append(dy/dx)
    ngrad = [ngrad[0]] + ngrad + [ngrad[-1]]
    return ngrad

num_grad = numerical_gradient(x, loss)


fig, ax1 = plt.subplots()
ax1.plot(x, grad, color='b', label='Gradient (JJ)')
ax1.plot(x, loss, color='r', label='Loss')

ax2 = ax1.twinx()
ax2.plot(x, num_grad, color='g', label='Numerical Gradient (Loss)', marker='x')

fig.legend(loc='lower right')

plt.show()

10
{'params': [0.15, 0.25], 'loss': 4.2443021567376805e+21, 'grad': [-2.3847288294865972e+27, -7.169382588432784e+27], 'loss_jj': -15008029652597.537, 'loss_sa': 9.67390442690232e+21, 'loss_ma': -1.7959092400286825e+21, 'grad_jj': [-7.741029104912324e+23, -3.782726742882062e+25], 'grad_sa': [-9.54486396805414e+26, -3.2981256972162167e+27], 'grad_ma': [-5.779086604935287e+27, -1.667092759427353e+28], 'intMetals': 141654703169.90643, 'intDielectric': 24716478551.30371, 'index': 0, 'w_jj': 0.5, 'w_sa': 0.5}


In [22]:
print(records[0].keys())

dict_keys(['params', 'loss', 'grad', 'loss_jj', 'loss_sa', 'loss_ma', 'grad_jj', 'grad_sa', 'grad_ma', 'intMetals', 'intDielectric', 'index', 'w_jj', 'w_sa'])


In [20]:

# epr_type = "sa"
# epr_type = "jj"
epr_type = "ma"



x = [r['params'][0] for r in records]

ngrad = [r[f"grad_{epr_type}"][0]for r in records]

loss = [r[f"loss_{epr_type}"]for r in records]

def numerical_gradient(x, y):
    grad = []
    for i in range(1, len(y)-1):
        dy = y[i+1] - y[i-1]
        dx = x[i+1] - x[i-1]
        try:
            grad.append(dy/dx)
        except ZeroDivisionError:
            grad.append(0.0)
    grad = [grad[0]] + grad + [grad[-1]]
    return grad

num_grad = numerical_gradient(x, loss)

def rescale(data):
    min_data = min(data)
    max_data = max(data)
    if max_data - min_data == 0:
        return [0.0 for d in data]
    return [(d - min_data) / (max_data - min_data) for d in data]

grad_rescale = rescale(ngrad)
loss_rescale = rescale(loss)
num_grad_rescale = rescale(num_grad)



fig, ax1 = plt.subplots()
ax1.plot(x, grad_rescale, color='b', label=f'Gradient ({epr_type})')
ax1.plot(x, loss_rescale, color='r', label='Loss')

ax2 = ax1.twinx()
ax2.plot(x, num_grad_rescale, color='g', label='Numerical Gradient (Loss)', marker='x')
plt.title(f'Width sweep at height{y_vals[offset]} - {epr_type}')
fig.legend(loc='lower right')
plt.show()


In [ ]:
# 
sa_estimate = [r["loss_sa"] for r in records]
ma_estimate = [r["loss_ma"] for r in records]

#comsol exact values
sa_exact = [r["intDielectric"] for r in records]
ma_exact = [r["intMetals"] for r in records]




4
dict_keys(['params', 'loss', 'grad', 'loss_jj', 'loss_sa', 'loss_ma', 'grad_jj', 'grad_sa', 'grad_ma'])
Computed 1 numerical gradients
